# Getting started

In this notebook, we illustrate how to use the Neural News Recommendation with Multi-Head Self-Attention ([NRMS](https://aclanthology.org/D19-1671/)). The implementation is taken from the [recommenders](https://github.com/recommenders-team/recommenders) repository. We have simply stripped the model to keep it cleaner.

We use a small dataset, which is downloaded from [recsys.eb.dk](https://recsys.eb.dk/). All the datasets are stored in the folder path ```~/ebnerd_data/*```.

## Load functionality

In [1]:
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import tensorflow as tf
import polars as pl
from tensorflow.python.client import device_lib
import numpy as np

from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_LABELS_COL,
    DEFAULT_TITLE_COL,
    DEFAULT_USER_COL,
    DEFAULT_IMPRESSION_TIMESTAMP_COL,
    DEFAULT_HISTORY_IMPRESSION_TIMESTAMP_COL,
    DEFAULT_ARTICLE_ID_COL
)

from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    add_known_user_column,
    add_prediction_scores,
    truncate_history,
)
from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
from ebrec.utils._articles import convert_text2encoding_with_transformers
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._articles import create_article_id_to_value_mapping
from ebrec.utils._nlp import get_transformers_word_embeddings
from ebrec.utils._python import write_submission_file, rank_predictions_by_score

from ebrec.models.newsrec.dataloader import NRMSDataLoader
from ebrec.models.newsrec.model_config import hparams_nrms
from ebrec.models.newsrec import NRMSModel, NRMSWrapper

2024-11-28 18:56:59.591421: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-28 18:56:59.611818: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-28 18:56:59.724020: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 18:56:59.724055: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 18:56:59.724520: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
# List all physical devices
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

physical_devices = tf.config.list_physical_devices('GPU')
print("Available devices:", physical_devices)

Available devices: []


2024-11-28 18:57:03.335174: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Load dataset

In [3]:
def ebnerd_from_path(path: Path, history_size: int = 30) -> pl.DataFrame:
    """
    Load ebnerd - function
    """
    df_history = (
        pl.scan_parquet(path.joinpath("history.parquet"))
        .select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL,DEFAULT_HISTORY_IMPRESSION_TIMESTAMP_COL)
        .pipe(
            truncate_history,
            column=DEFAULT_HISTORY_ARTICLE_ID_COL,
            history_size=history_size,
            padding_value=0,
            enable_warning=False,
        )
    )
    df_behaviors = (
        pl.scan_parquet(path.joinpath("behaviors.parquet"))
        .collect()
        .pipe(
            slice_join_dataframes,
            df2=df_history.collect(),
            on=DEFAULT_USER_COL,
            how="left",
        )
    )
    
    return df_behaviors

### Generate labels
We sample a few just to get started. For testset we just make up a dummy column with 0 and 1 - this is not the true labels.

In [4]:
PATH = Path("/dtu/blackhole/14/155764/DeepL-Project-Corn2/ebnerd-benchmark-copy/ebnerd_data").expanduser()
DATASPLIT = "ebnerd_small" # REMEMBER if change to change make_embedding_artifacts.ipynb file (embeddings)
# DATASPLIT = "ebnerd__testset"
DUMP_DIR = PATH.joinpath("dump_artifacts")
DUMP_DIR.mkdir(exist_ok=True, parents=True)

In this example we sample the dataset, just to keep it smaller. Also, one can simply add the testset similary to the validation.

In [ ]:
COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_IMPRESSION_TIMESTAMP_COL,
]
HISTORY_SIZE = 20 #20
FRACTION =0.01  # 0.01 #Ratio of Dataset!

df_train = (
                                            #REMEMBER #"train"
    ebnerd_from_path(PATH.joinpath(DATASPLIT, "train"), history_size=HISTORY_SIZE)
    .select(COLUMNS)
    .pipe(
        sampling_strategy_wu2019,
        npratio=4,
        shuffle=True,
        with_replacement=True,
        seed=123,
    )
    .pipe(create_binary_labels_column)
    .sample(fraction=FRACTION)
)
# =>
df_validation = (
    ebnerd_from_path(PATH.joinpath(DATASPLIT, "validation"), history_size=HISTORY_SIZE)
    .select(COLUMNS)
    .pipe(create_binary_labels_column)
    .sample(fraction=FRACTION)
)
print(df_train.head(2))
print(df_validation.head(2))

shape: (2, 7)
┌─────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┐
│ user_id ┆ article_id_f ┆ article_ids_ ┆ article_ids_ ┆ impression_i ┆ impression_t ┆ labels      │
│ ---     ┆ ixed         ┆ inview       ┆ clicked      ┆ d            ┆ ime          ┆ ---         │
│ u32     ┆ ---          ┆ ---          ┆ ---          ┆ ---          ┆ ---          ┆ list[i8]    │
│         ┆ list[i32]    ┆ list[i64]    ┆ list[i64]    ┆ u32          ┆ datetime[μs] ┆             │
╞═════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪═════════════╡
│ 1007040 ┆ [9769828,    ┆ [9771787,    ┆ [9771787]    ┆ 253110032    ┆ 2023-05-19   ┆ [1, 0, … 0] │
│         ┆ 9769679, …   ┆ 9772061, …   ┆              ┆              ┆ 10:06:29     ┆             │
│         ┆ 9557348]     ┆ 9772032]     ┆              ┆              ┆              ┆             │
│ 961876  ┆ [9769580,    ┆ [9775361,    ┆ [9774595]    ┆ 281008021    ┆ 2023-

## Load articles

In [6]:
df_articles = pl.read_parquet(PATH.joinpath(DATASPLIT, "articles.parquet"))
df_articles.head(2)

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var i…","""Politiet frygt…",2023-06-29 06:20:33,false,"""Sagen om den ø…",2006-08-31 08:06:45,[3150850],"""article_defaul…","""https://ekstra…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars …","""Biografgængern…",2023-06-29 06:20:35,false,"""Vatikanet har …",2006-05-21 16:57:00,[3006712],"""article_defaul…","""https://ekstra…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""


## hourly difference between published and viewed article

In [7]:
## NEW

from sklearn.preprocessing import StandardScaler

# Convert polars DataFrame to pandas
df_train = df_train.to_pandas()

# Create a mapping dictionary from article_id to last_modified_time
article_time_dict = df_articles.select(
    "article_id", 
    "published_time"
).to_dict(as_series=False)
article_time_dict = dict(zip(
    article_time_dict["article_id"], 
    article_time_dict["published_time"]
))

# Create a function to map article IDs to their timestamps
def get_article_times(article_ids):
    return [article_time_dict.get(aid, None) for aid in article_ids]

# Add new column with the published-time
df_train["inview_article_times"] = df_train["article_ids_inview"].apply(get_article_times)

#add new column with the last publish_time for the clicked article
df_train["clicked_article_time"] = df_train["article_ids_clicked"].apply(get_article_times)

# Create a function to calculate hour differences
def calculate_hour_differences(impression_time, article_times):
        # If article_times is a single value (for clicked articles)
    if not isinstance(article_times, list):
        if article_times is None:
            return None
        return (impression_time - article_times).total_seconds() / 3600
    
    # If article_times is a list (for inview articles)
    differences = [(impression_time - article_time).total_seconds() / 3600 
                  if article_time is not None else None 
                  for article_time in article_times]
    return differences

# Use for inview articles
df_train['inview_hour_differences'] = df_train.apply(
    lambda row: calculate_hour_differences(row['impression_time'], row['inview_article_times']), 
    axis=1
)

# # Use for clicked article
# df_train['clicked_hour_difference'] = df_train.apply(
#    lambda row: calculate_hour_differences(row['impression_time'], row['clicked_article_time']), 
#    axis=1
# )


#drop columns with the time
df_train = df_train.drop(['inview_article_times', 'clicked_article_time','impression_time'], axis=1)

df_train = pl.from_pandas(df_train)

df_train.head(2)

user_id,article_id_fixed,article_ids_inview,article_ids_clicked,impression_id,labels,inview_hour_differences
u32,list[i32],list[i64],list[i64],u32,list[i8],list[f64]
1007040,"[9769828, 9769679, … 9557348]","[9771787, 9772061, … 9772032]",[9771787],253110032,"[1, 0, … 0]","[5.263611, 4.488889, … 4.512778]"
961876,"[9769580, 9768850, … 9770867]","[9775361, 9774595, … 9521144]",[9774595],281008021,"[0, 1, … 0]","[8.388333, 7.896944, … 4361.339722]"


In [8]:
# Convert polars DataFrame to pandas
df_validation = df_validation.to_pandas()

# Create a mapping dictionary from article_id to last_modified_time
article_time_dict = df_articles.select(
    "article_id", 
    "published_time"
).to_dict(as_series=False)
article_time_dict = dict(zip(
    article_time_dict["article_id"], 
    article_time_dict["published_time"]
))

# Create a function to map article IDs to their timestamps
def get_article_times(article_ids):
    return [article_time_dict.get(aid, None) for aid in article_ids]

# Add new column with the published-time
df_validation["inview_article_times"] = df_validation["article_ids_inview"].apply(get_article_times)

#add new column with the last publish_time for the clicked article
df_validation["clicked_article_time"] = df_validation["article_ids_clicked"].apply(get_article_times)

# Create a function to calculate hour differences
def calculate_hour_differences(impression_time, article_times):
        # If article_times is a single value (for clicked articles)
    if not isinstance(article_times, list):
        if article_times is None:
            return None
        return (impression_time - article_times).total_seconds() / 3600
    
    # If article_times is a list (for inview articles)
    differences = [(impression_time - article_time).total_seconds() / 3600 
                  if article_time is not None else None 
                  for article_time in article_times]
    return differences

# Use for inview articles
df_validation['inview_hour_differences'] = df_validation.apply(
    lambda row: calculate_hour_differences(row['impression_time'], row['inview_article_times']), 
    axis=1
)

# # Use for clicked article -- might be leaky??
# df_validation['clicked_hour_difference'] = df_validation.apply(
#    lambda row: calculate_hour_differences(row['impression_time'], row['clicked_article_time']), 
#    axis=1
# )

#drop columns with the time
df_validation = df_validation.drop(['inview_article_times', 'clicked_article_time','impression_time'], axis=1)


df_validation = pl.from_pandas(df_validation)

df_validation.head(2)

user_id,article_id_fixed,article_ids_inview,article_ids_clicked,impression_id,labels,inview_hour_differences
u32,list[i32],list[i32],list[i32],u32,list[i8],list[f64]
1817139,"[9777182, 9779289, … 9779723]","[9785310, 9785471, … 9785426]",[9785426],548917191,"[0, 0, … 1]","[3.602222, 1.598056, … 1.841111]"
801768,"[9776337, 9778007, … 9778661]","[9786066, 9785017, … 9785829]",[9785829],490802699,"[0, 0, … 1]","[2.438056, 7.098889, … 1.674167]"


## Init model using HuggingFace's tokenizer and wordembedding
In the original implementation, they use the GloVe embeddings and tokenizer. To get going fast, we'll use a multilingual LLM from Hugging Face. 
Utilizing the tokenizer to tokenize the articles and the word-embedding to init NRMS.


In [9]:
# TRANSFORMER_MODEL_NAME = "FacebookAI/xlm-roberta-base"
TRANSFORMER_MODEL_NAME = "FacebookAI/xlm-roberta-large"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30



# LOAD HUGGINGFACE:
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# We'll init the word embeddings using the
word2vec_embedding = get_transformers_word_embeddings(transformer_model)
#


df_articles, cat_cal = concat_str_columns(df_articles, columns=TEXT_COLUMNS_TO_USE)
df_articles, token_col_title = convert_text2encoding_with_transformers(
    df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)

# =>
article_mapping = create_article_id_to_value_mapping(
    df=df_articles, value_col=token_col_title
)


/dtu/blackhole/14/155764/DeepL-Project-Corn2/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
word2vec_embedding

array([[-0.0562439 , -0.02876282, -0.07940674, ...,  0.02944946,
         0.0267334 , -0.02970886],
       [-0.0042305 ,  0.0026474 ,  0.0078125 , ..., -0.00432587,
         0.00658798, -0.00568771],
       [-0.0147171 ,  0.01287842, -0.10516357, ...,  0.11804199,
         0.13781738,  0.203125  ],
       ...,
       [ 0.17944336,  0.1887207 ,  0.24584961, ..., -0.01290894,
         0.18457031, -0.15307617],
       [ 0.05737305, -0.03430176, -0.07635498, ...,  0.0725708 ,
         0.06689453,  0.06781006],
       [ 0.02632141,  0.01670837, -0.04385376, ...,  0.01147461,
         0.08642578,  0.06854248]], dtype=float32)

# Initiate the dataloaders
In the implementations we have disconnected the models and data. Hence, you should built a dataloader that fits your needs.

In [10]:
train_dataloader = NRMSDataLoader(
    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=128,
)
val_dataloader = NRMSDataLoader(
    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=64,
)

## Train the model


In [11]:
# List all physical devices
physical_devices = tf.config.list_physical_devices('GPU')
print("Available devices:", physical_devices)
# import torch.nn as nn
# print(torch.cuda.is_available())

Available devices: []


In [ ]:
# import os
# MODEL_NAME = "NRMS"
# LOG_DIR = f"downloads/runs/{MODEL_NAME}"
# WEIGHTS_DIR = f"downloads/data/state_dict/{MODEL_NAME}"
# MODEL_WEIGHTS = f"{WEIGHTS_DIR}/weights.weights.h5"

# os.makedirs(LOG_DIR, exist_ok=True)
# os.makedirs(WEIGHTS_DIR, exist_ok=True)

# # CALLBACKS
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
# modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(
#     filepath=MODEL_WEIGHTS, save_best_only=True, save_weights_only=True, verbose=1
# )

# # hparams_nrms.history_size = HISTORY_SIZE
# # model = NRMSModel(
# #     hparams=hparams_nrms,
# #     word2vec_embedding=word2vec_embedding,
# #     # precomputed_embeddings=article_embeddings,
# #     seed=42,
# # )
# # print("starting fitting")

# # hist = model.model.fit(
# #     train_dataloader,
# #     validation_data=val_dataloader,
# #     epochs=1,
# #     callbacks=[early_stopping, modelcheckpoint]#tensorboard_callback
# # )
# # print("load weigths")
# # _ = model.model.load_weights(filepath=MODEL_WEIGHTS)


In [ ]:
# # from ebrec.models.newsrec.nrmspy import NRMSModel, NRMSWrapper 
# # import torch
# os.makedirs(LOG_DIR, exist_ok=True)
# os.makedirs(WEIGHTS_DIR, exist_ok=True)

# # Create a custom ModelCheckpoint for PyTorch
# class PyTorchModelCheckpoint:
#     def __init__(self, filepath, save_best_only=True, save_weights_only=True, verbose=1):
#         self.filepath = filepath
#         self.save_best_only = save_best_only
#         self.save_weights_only = save_weights_only
#         self.verbose = verbose
#         self.best_val_loss = float('inf')
    
#     def on_epoch_end(self, epoch, logs=None):
#         val_loss = logs.get('val_loss', None)
#         if val_loss is None:
#             return
        
#         if self.save_best_only:
#             if val_loss < self.best_val_loss:
#                 if self.verbose:
#                     print(f'\nValidation loss improved from {self.best_val_loss:.5f} to {val_loss:.5f}')
#                 self.best_val_loss = val_loss
#                 torch.save(model.model.state_dict(), self.filepath)
#         else:
#             torch.save(model.model.state_dict(), self.filepath)

# # CALLBACKS
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
# modelcheckpoint = PyTorchModelCheckpoint(
#     filepath=MODEL_WEIGHTS,
#     save_best_only=True,
#     save_weights_only=True,
#     verbose=1
# )

# # # Update hyperparameters if needed
# # hparams_nrms.head_num = 20  # number of attention heads
# # hparams_nrms.head_dim = 20  # dimension of each head (total dimension will be head_num * head_dim)
# # hparams_nrms.attention_hidden_dim = hparams_nrms.head_num * hparams_nrms.head_dim
# # hparams_nrms.dropout = 0.2
# # hparams_nrms.title_size = 30  # should match your MAX_TITLE_LENGTH

# # # Initialize model
# # hparams_nrms.history_size = HISTORY_SIZE
# # pytorch_model = NRMSModel(
# #     hparams=hparams_nrms,
# #     word2vec_embedding=word2vec_embedding,
# #     seed=42,
# # )
# # model = NRMSWrapper(pytorch_model)

# # # Training
# # hist = model.fit(
# #     train_dataloader,
# #     validation_data=val_dataloader,
# #     epochs=5,
# #     callbacks=[modelcheckpoint]
# # )

# # # Load weights using the wrapper
# # model.load_weights(filepath=MODEL_WEIGHTS)

# # # Get predictions
# # pred_validation = model.predict(val_dataloader)

In [ ]:
# import os
# import torch
# from ebrec.models.newsrec.nrmspy import NRMSModel, NRMSWrapper  # Add this import

# MODEL_NAME = "NRMS"
# LOG_DIR = f"downloads/runs/{MODEL_NAME}"
# WEIGHTS_DIR = f"downloads/data/state_dict/{MODEL_NAME}"
# MODEL_WEIGHTS = f"{WEIGHTS_DIR}/weights.pt"

# os.makedirs(LOG_DIR, exist_ok=True)
# os.makedirs(WEIGHTS_DIR, exist_ok=True)

# # Create base model
# base_model = NRMSModel(
#     hparams=hparams_nrms,
#     word2vec_embedding=word2vec_embedding,
#     seed=42,
# )

# # Wrap model with NRMSWrapper
# model = NRMSWrapper(
#     model=base_model,
#     device='cuda' if torch.cuda.is_available() else 'cpu'
# )

# print("starting fitting")

# # Use wrapper's fit method
# hist = model.fit(
#     train_dataloader,
#     validation_data=val_dataloader,
#     epochs=1,
#     callbacks=[early_stopping, modelcheckpoint]
# )

# print("load weights")
# model.load_weights(filepath=MODEL_WEIGHTS)

In [ ]:
# import os
# from torch.utils.tensorboard import SummaryWriter
# from ebrec.models.newsrec.nrmspy import NRMSModel, NRMSWrapper
# import torch

# # Setup directories
# MODEL_NAME = "NRMS"
# LOG_DIR = f"downloads/runs/{MODEL_NAME}"
# WEIGHTS_DIR = f"downloads/data/state_dict/{MODEL_NAME}"
# MODEL_WEIGHTS = f"{WEIGHTS_DIR}/weights.pt"

# os.makedirs(LOG_DIR, exist_ok=True)
# os.makedirs(WEIGHTS_DIR, exist_ok=True)

# # Create tensorboard writer
# writer = SummaryWriter(LOG_DIR)

# # Setup ModelCheckpoint
# class PyTorchModelCheckpoint:
#     def __init__(self, filepath, save_best_only=True, save_weights_only=True, verbose=1):
#         self.filepath = filepath
#         self.save_best_only = save_best_only
#         self.save_weights_only = save_weights_only
#         self.verbose = verbose
#         self.best_val_loss = float('inf')
    
#     def on_epoch_end(self, epoch, logs=None):
#         val_loss = logs.get('val_loss', None)
#         if val_loss is None:
#             return
        
#         if self.save_best_only:
#             if val_loss < self.best_val_loss:
#                 if self.verbose:
#                     print(f'\nValidation loss improved from {self.best_val_loss:.5f} to {val_loss:.5f}')
#                 self.best_val_loss = val_loss
#                 torch.save(model.state_dict(), self.filepath)
#         else:
#             torch.save(model.state_dict(), self.filepath)

# # Initialize callback
# modelcheckpoint = PyTorchModelCheckpoint(
#     filepath=MODEL_WEIGHTS,
#     save_best_only=True,
#     save_weights_only=True,
#     verbose=1
# )

# # Create model and wrapper
# model = NRMSModel(
#     hparams=hparams_nrms,
#     word2vec_embedding=word2vec_embedding,
#     seed=42
# )
# wrapper = NRMSWrapper(model)

# # Train
# wrapper.fit(
#     train_dataloader,
#     validation_data=val_dataloader,
#     epochs=5,
#     callbacks=[modelcheckpoint],
#     writer=writer  # Pass writer to fit method
# )

# # Close writer
# writer.close()

# # Load best weights and predict
# wrapper.load_weights(MODEL_WEIGHTS)
# predictions = wrapper.predict(val_dataloader)

In [ ]:
# # Create model with Keras-like interface
# from ebrec.models.newsrec.nrmspy import NRMSModel, NRMSWrapper, KerasLikeModel

# import os
# from torch.utils.tensorboard import SummaryWriter
# from ebrec.models.newsrec.nrmspy import NRMSModel, NRMSWrapper
# import torch

# # Setup directories
# MODEL_NAME = "NRMS"
# LOG_DIR = f"downloads/runs/{MODEL_NAME}"
# WEIGHTS_DIR = f"downloads/data/state_dict/{MODEL_NAME}"
# MODEL_WEIGHTS = f"{WEIGHTS_DIR}/weights.pt"

# os.makedirs(LOG_DIR, exist_ok=True)
# os.makedirs(WEIGHTS_DIR, exist_ok=True)

# # Create tensorboard writer
# writer = SummaryWriter(LOG_DIR)

# # Setup ModelCheckpoint
# class PyTorchModelCheckpoint:
#     def __init__(self, filepath, save_best_only=True, save_weights_only=True, verbose=1):
#         self.filepath = filepath
#         self.save_best_only = save_best_only
#         self.save_weights_only = save_weights_only
#         self.verbose = verbose
#         self.best_val_loss = float('inf')
    
#     def on_epoch_end(self, epoch, logs=None):
#         val_loss = logs.get('val_loss', None)
#         if val_loss is None:
#             return
        
#         if self.save_best_only:
#             if val_loss < self.best_val_loss:
#                 if self.verbose:
#                     print(f'\nValidation loss improved from {self.best_val_loss:.5f} to {val_loss:.5f}')
#                 self.best_val_loss = val_loss
#                 torch.save(model.state_dict(), self.filepath)
#         else:
#             torch.save(model.state_dict(), self.filepath)

# # Initialize callback
# modelcheckpoint = PyTorchModelCheckpoint(
#     filepath=MODEL_WEIGHTS,
#     save_best_only=True,
#     save_weights_only=True,
#     verbose=1
# )

# pytorch_model = NRMSModel(hparams=hparams_nrms, word2vec_embedding=word2vec_embedding, seed=42)
# keras_like_model = KerasLikeModel(pytorch_model)

# # Compile model
# keras_like_model.compile(optimizer='adam', loss='categorical_crossentropy')

# # Train model
# keras_like_model.fit(
#     train_dataloader,
#     validation_data=val_dataloader,
#     epochs=5,
#     callbacks=[modelcheckpoint]
# )

In [12]:
import os
from tqdm.notebook import tqdm


MODEL_NAME = "NRMS"
LOG_DIR = f"downloads/runs/{MODEL_NAME}"
WEIGHTS_DIR = f"downloads/data/state_dict/{MODEL_NAME}"
MODEL_WEIGHTS = f"{WEIGHTS_DIR}/weights.pt"

os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(WEIGHTS_DIR, exist_ok=True)

# Create a custom ModelCheckpoint for PyTorch
class PyTorchModelCheckpoint:
    def __init__(self, filepath, model_wrapper=None, save_best_only=True, save_weights_only=True, verbose=1):
        self.filepath = filepath
        self.model_wrapper = model_wrapper  # Store the model wrapper reference
        self.save_best_only = save_best_only
        self.save_weights_only = save_weights_only
        self.verbose = verbose
        self.best_val_loss = float('inf')
    
    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss', None)
        if val_loss is None:
            return
        
        if self.save_best_only:
            if val_loss < self.best_val_loss:
                if self.verbose:
                    print(f'\nValidation loss improved from {self.best_val_loss:.5f} to {val_loss:.5f}')
                self.best_val_loss = val_loss
                # Use the model_wrapper reference
                self.model_wrapper.save_weights(self.filepath)
        else:
            self.model_wrapper.save_weights(self.filepath)

# Initialize model first
hparams_nrms.history_size = HISTORY_SIZE
pytorch_model = NRMSModel(
    hparams=hparams_nrms,
    word2vec_embedding=word2vec_embedding,
    seed=42,
)
model = NRMSWrapper(pytorch_model)

# Then create the callback with the model reference
modelcheckpoint = PyTorchModelCheckpoint(
    filepath=MODEL_WEIGHTS,
    model_wrapper=model,  # Pass the model wrapper
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Training
hist = model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=1,
    callbacks=[modelcheckpoint]
)

# Load weights using the wrapper
model.load_weights(filepath=MODEL_WEIGHTS)

# Get predictions
pred_validation = model.predict(val_dataloader)

NRMSWrapper init
True


Epoch 1/1 [Valid]: 100%|██████████| 39/39 [00:07<00:00,  5.07it/s, loss=0.7819]


Epoch 1 - Train Loss: 0.5436, Val Loss: 0.3809

Validation loss improved from inf to 0.38093


/dtu/blackhole/14/155764/DeepL-Project-Corn2/ebnerd-benchmark-copy/src/ebrec/models/newsrec/nrmspy_1.py:153: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_st

In [ ]:
#TODO Attempt at new

# import os
# from tqdm.notebook import tqdm
# from transformers import AutoModel, AutoTokenizer

# # Define constants
# MODEL_NAME = "NRMS"
# LOG_DIR = f"downloads/runs/{MODEL_NAME}"
# WEIGHTS_DIR = f"downloads/data/state_dict/{MODEL_NAME}"
# MODEL_WEIGHTS = f"{WEIGHTS_DIR}/weights.pt"
# TRANSFORMER_MODEL_NAME = "FacebookAI/xlm-roberta-large"

# # Create necessary directories
# os.makedirs(LOG_DIR, exist_ok=True)
# os.makedirs(WEIGHTS_DIR, exist_ok=True)

# # Load transformer model and tokenizer
# transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
# transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# # Create a custom ModelCheckpoint for PyTorch
# class PyTorchModelCheckpoint:
#     def __init__(self, filepath, model_wrapper=None, save_best_only=True, save_weights_only=True, verbose=1):
#         self.filepath = filepath
#         self.model_wrapper = model_wrapper  # Store the model wrapper reference
#         self.save_best_only = save_best_only
#         self.save_weights_only = save_weights_only
#         self.verbose = verbose
#         self.best_val_loss = float('inf')
    
#     def on_epoch_end(self, epoch, logs=None):
#         val_loss = logs.get('val_loss', None)
#         if val_loss is None:
#             return
        
#         if self.save_best_only:
#             if val_loss < self.best_val_loss:
#                 if self.verbose:
#                     print(f'\nValidation loss improved from {self.best_val_loss:.5f} to {val_loss:.5f}')
#                 self.best_val_loss = val_loss
#                 # Use the model_wrapper reference
#                 self.model_wrapper.save_weights(self.filepath)
#         else:
#             self.model_wrapper.save_weights(self.filepath)

# # Define model hyperparameters
# hparams_nrms = {
#     "head_num": 8,
#     "head_dim": 64,
#     "title_size": MAX_TITLE_LENGTH,
#     "history_size": HISTORY_SIZE,
#     "dropout": 0.2 #NOT USED
# } # from gpt

# # init NRMS model
# pytorch_model = NRMSModel(
#     hparams=hparams_nrms,
#     transformer_model=transformer_model,
#     transformer_tokenizer=transformer_tokenizer,
#     seed=42
# )
# model = NRMSWrapper(pytorch_model)

# # Create the checkpoint callback
# modelcheckpoint = PyTorchModelCheckpoint(
#     filepath=MODEL_WEIGHTS,
#     model_wrapper=model,  # Pass the model wrapper
#     save_best_only=True,
#     save_weights_only=True,
#     verbose=1
# )

# # fit
# hist = model.fit(
#     train_dataloader,
#     validation_data=val_dataloader,
#     epochs=1,
#     callbacks=[modelcheckpoint]
# )

# # Load weights using the wrapper
# model.load_weights(filepath=MODEL_WEIGHTS)

# # Get predictions
# pred_validation = model.predict(val_dataloader)

# Saving / loading model because hpc annoying

In [19]:
# MODEL_FILE = f"downloads/models/{MODEL_NAME}.h5" 

# # Save the model after training
# print("Saving the model...")
# os.makedirs(os.path.dirname(MODEL_FILE), exist_ok=True)
# model.model.save(MODEL_FILE)  # Save the full model (architecture + weights)
# print(f"Model saved at {MODEL_FILE}")

##LOAD SAVED MODEL
# from tensorflow.keras.models import load_model

# # Load the saved model
# print(f"Loading the model from {MODEL_FILE}...")
# model.model = load_model(MODEL_FILE)
# print("Model loaded successfully.")


# Example how to compute some metrics:

In [20]:
# pred_validation = model.scorer.predict(val_dataloader)

## Add the predictions to the dataframe

In [21]:
df_validation = add_prediction_scores(df_validation, pred_validation.tolist()).pipe(
    add_known_user_column, known_users=df_train[DEFAULT_USER_COL]
)
df_validation.head(2)

user_id,article_id_fixed,article_ids_inview,article_ids_clicked,impression_id,labels,inview_hour_differences,scores,is_known_user
u32,list[i32],list[i32],list[i32],u32,list[i8],list[f64],list[f64],bool
22548,"[9773295, 9769504, … 9776929]","[9783865, 9784710, … 9784696]",[9784696],96791,"[0, 0, … 1]","[6.831111, 6.814444, … 8.163333]","[0.056111, 0.245147, … 0.254644]",true
22548,"[9773295, 9769504, … 9776929]","[9784281, 9784583, … 9782726]",[9784281],96798,"[1, 0, … 0]","[12.075, 9.953889, … 0.53]","[0.272449, 0.102848, … 0.137341]",true


### Compute metrics

In [22]:
metrics = MetricEvaluator(
    labels=df_validation["labels"].to_list(),
    predictions=df_validation["scores"].to_list(),
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()

<MetricEvaluator class>: 
 {
    "auc": 0.5429764670849283,
    "mrr": 0.3383194556757905,
    "ndcg@5": 0.37634755125872577,
    "ndcg@10": 0.4549911716551946
}

## Make submission file

In [23]:
df_validation = df_validation.with_columns(
    pl.col("scores")
    .map_elements(lambda x: list(rank_predictions_by_score(x)))
    .alias("ranked_scores")
)
df_validation.head(2)

user_id,article_id_fixed,article_ids_inview,article_ids_clicked,impression_id,labels,inview_hour_differences,scores,is_known_user,ranked_scores
u32,list[i32],list[i32],list[i32],u32,list[i8],list[f64],list[f64],bool,list[i64]
22548,"[9773295, 9769504, … 9776929]","[9783865, 9784710, … 9784696]",[9784696],96791,"[0, 0, … 1]","[6.831111, 6.814444, … 8.163333]","[0.056111, 0.245147, … 0.254644]",true,"[5, 2, … 1]"
22548,"[9773295, 9769504, … 9776929]","[9784281, 9784583, … 9782726]",[9784281],96798,"[1, 0, … 0]","[12.075, 9.953889, … 0.53]","[0.272449, 0.102848, … 0.137341]",true,"[4, 18, … 15]"


This is using the validation, simply add the testset to your flow.

In [24]:
write_submission_file(
    impression_ids=df_validation[DEFAULT_IMPRESSION_ID_COL],
    prediction_scores=df_validation["ranked_scores"],
    path="downloads/predictions.txt",
)

0it [00:00, ?it/s]

244647it [00:10, 22954.92it/s]


Zipping downloads/predictions.txt to downloads/predictions.zip


# DONE 🚀